https://drive.google.com/drive/u/0/folders/1X_ag69ToGRCF32K2a099pgvvNMg0j8nq

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

In [ ]:
print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

In [ ]:
print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

In [ ]:
num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_0 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_0 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_0 = remaining_prices_0_A.loc[max_price_index_0, 'P_dam']
            min_price_index_0 = remaining_prices_0_B.loc[min_price_index_0, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_0/2, 'Price'] < current_Q_A.loc[max_price_index_0/2, 'Price']:
                  if min_price_index_0 < max_price_index_0:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
            
                    profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                    prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_0 > max_price_index_0:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
            
                        profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)
            
                        prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_0a = min(min_price_index_0, max_price_index_0)
            larger_index_0a = max(min_price_index_0, max_price_index_0)
            remaining_prices_0a = current_df[(current_df['P_dam'] > smaller_index_0a) & (current_df['P_dam'] < larger_index_0a)]
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0a) & (Q_A_Preds['P_dam'] < larger_index_0a)]
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0a) & (Q_B_Preds['P_dam'] < larger_index_0a)]

            if len(remaining_prices_0a) > 1:

                    max_price_index_0a = remaining_prices_0a_A['Price'].idxmax()
                    min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                    max_price_index_0a = remaining_prices_0a_A.loc[max_price_index_0a, 'P_dam']
                    min_price_index_0a = remaining_prices_0a_B.loc[min_price_index_0a, 'P_dam']
                    if current_Q_B.loc[min_price_index_0a/2, 'Price'] < current_Q_A.loc[max_price_index_0a/2, 'Price']:
                        if min_price_index_0a < max_price_index_0a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                    
                            profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                            prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_0a > max_price_index_0a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                    
                              profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)
                    
                              prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue     
                                
                                
                                
                                
                    smaller_index_0b = min(min_price_index_0a, max_price_index_0a)
                    larger_index_0b = max(min_price_index_0a, max_price_index_0a)
                    remaining_prices_0b = current_df[(current_df['P_dam'] > smaller_index_0b) & (current_df['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0b) & (Q_A_Preds['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0b) & (Q_B_Preds['P_dam'] < larger_index_0b)]

                    if len(remaining_prices_0b) > 1:

                            max_price_index_0b = remaining_prices_0b_A['Price'].idxmax()
                            min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                            max_price_index_0b = remaining_prices_0b_A.loc[max_price_index_0b, 'P_dam']
                            min_price_index_0b = remaining_prices_0b_B.loc[min_price_index_0b, 'P_dam']
                            if current_Q_B.loc[min_price_index_0b/2, 'Price'] < current_Q_A.loc[max_price_index_0b/2, 'Price']:
                                if min_price_index_0b < max_price_index_0b:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                    
                                    profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                    
                                elif min_price_index_0b > max_price_index_0b:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                    
                                      profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                      prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                    
                                
                                
 
                            smaller_index_0c = min(min_price_index_0b, max_price_index_0b)
                            larger_index_0c = max(min_price_index_0b, max_price_index_0b)
                            remaining_prices_0c = current_df[(current_df['P_dam'] > smaller_index_0c) & (current_df['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0c) & (Q_A_Preds['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0c) & (Q_B_Preds['P_dam'] < larger_index_0c)]

                            if len(remaining_prices_0c) > 1:

                                    max_price_index_0c = remaining_prices_0c_A['Price'].idxmax()
                                    min_price_index_0c = remaining_prices_0c_B['Price'].idxmin()
                                    max_price_index_0c = remaining_prices_0c_A.loc[max_price_index_0c, 'P_dam']
                                    min_price_index_0c = remaining_prices_0c_B.loc[min_price_index_0c, 'P_dam']
                                    if current_Q_B.loc[min_price_index_0c/2, 'Price'] < current_Q_A.loc[max_price_index_0c/2, 'Price']:
                                        if min_price_index_0c < max_price_index_0c:
                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                            charge_level += bottleneck_1
                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                            charge_level -= bottleneck_2
                    
                                            profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                            prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                    
                                        elif min_price_index_0c > max_price_index_0c:
                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                              charge_level -= bottleneck_2
                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                              charge_level += bottleneck_1
                    
                                              profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                              prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                                
                                        else:
                                             continue


                                    smaller_index_0d = min(min_price_index_0c, max_price_index_0c)
                                    larger_index_0d = max(min_price_index_0c, max_price_index_0c)
                                    remaining_prices_0d = current_df[(current_df['P_dam'] > smaller_index_0d) & (current_df['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0d) & (Q_A_Preds['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0d) & (Q_B_Preds['P_dam'] < larger_index_0d)]

                                    if len(remaining_prices_0d) > 1:

                                            max_price_index_0d = remaining_prices_0d_A['Price'].idxmax()
                                            min_price_index_0d = remaining_prices_0d_B['Price'].idxmin()
                                            max_price_index_0d = remaining_prices_0d_A.loc[max_price_index_0d, 'P_dam']
                                            min_price_index_0d = remaining_prices_0d_B.loc[min_price_index_0d, 'P_dam']
                                            if current_Q_B.loc[min_price_index_0d/2, 'Price'] < current_Q_A.loc[max_price_index_0d/2, 'Price']:
                                                if min_price_index_0d < max_price_index_0d:
                                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                    charge_level += bottleneck_1
                                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                    charge_level -= bottleneck_2
                    
                                                    profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                    prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                    
                                                elif min_price_index_0d > max_price_index_0d:
                                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                      charge_level -= bottleneck_2
                                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                      charge_level += bottleneck_1
                    
                                                      profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                      prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                                
                                                else:
                                                     continue



                                            smaller_index_0e = min(min_price_index_0d, max_price_index_0d)
                                            larger_index_0e = max(min_price_index_0d, max_price_index_0d)
                                            remaining_prices_0e = current_df[(current_df['P_dam'] > smaller_index_0e) & (current_df['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0e) & (Q_A_Preds['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0e) & (Q_B_Preds['P_dam'] < larger_index_0e)]

                                            if len(remaining_prices_0e) > 1:

                                                    max_price_index_0e = remaining_prices_0e_A['Price'].idxmax()
                                                    min_price_index_0e = remaining_prices_0e_B['Price'].idxmin()
                                                    max_price_index_0e = remaining_prices_0e_A.loc[max_price_index_0e, 'P_dam']
                                                    min_price_index_0e = remaining_prices_0e_B.loc[min_price_index_0e, 'P_dam']
                                                    if current_Q_B.loc[min_price_index_0e/2, 'Price'] < current_Q_A.loc[max_price_index_0e/2, 'Price']:
                                                        if min_price_index_0e < max_price_index_0e:
                                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                            charge_level += bottleneck_1
                                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                            charge_level -= bottleneck_2
                    
                                                            profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                            prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                    
                                                        elif min_price_index_0e > max_price_index_0e:
                                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                              charge_level -= bottleneck_2
                                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                              charge_level += bottleneck_1
                    
                                                              profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)
                    
                                                              prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                                
                                                        else:
                                                             continue









                                            
        if len(remaining_prices_1) > 1:

                max_price_index_1 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_1 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_1, 'Price'] < current_Q_A_bm.loc[max_price_index_1, 'Price']:
                      if min_price_index_1 < max_price_index_1:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      elif min_price_index_1 > max_price_index_1:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
            
                             profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)
            
                             prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      else:
                            continue                
                        

        
                smaller_index_1a = min(min_price_index_1, max_price_index_1)
                larger_index_1a = max(min_price_index_1, max_price_index_1)
                remaining_prices_1a = current_df_bm[(current_df_bm.index > smaller_index_1a) & (current_df_bm.index < larger_index_1a)]
                remaining_prices_1a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1a) & (Q_A_Preds_bm.index < larger_index_1a)]
                remaining_prices_1a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1a) & (Q_B_Preds_bm.index < larger_index_1a)]

                if len(remaining_prices_1a) > 1:
                        max_price_index_1a = remaining_prices_1a_A['Price'].idxmax()
                        min_price_index_1a = remaining_prices_1a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_1a, 'Price'] < current_Q_A_bm.loc[max_price_index_1a, 'Price']:
                            if min_price_index_1a < max_price_index_1a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1a > max_price_index_1a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                        larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                        remaining_prices_1b = current_df_bm[(current_df_bm.index > smaller_index_1b) & (current_df_bm.index < larger_index_1b)]
                        remaining_prices_1b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1b) & (Q_A_Preds_bm.index < larger_index_1b)]
                        remaining_prices_1b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1b) & (Q_B_Preds_bm.index < larger_index_1b)]

                        if len(remaining_prices_1b) > 1:
                                max_price_index_1b = remaining_prices_1b_A['Price'].idxmax()
                                min_price_index_1b = remaining_prices_1b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_1b, 'Price'] < current_Q_A_bm.loc[max_price_index_1b, 'Price']:
                                    if min_price_index_1b < max_price_index_1b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_1b > max_price_index_1b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                                larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                                remaining_prices_1c = current_df_bm[(current_df_bm.index > smaller_index_1c) & (current_df_bm.index < larger_index_1c)]
                                remaining_prices_1c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1c) & (Q_A_Preds_bm.index < larger_index_1c)]
                                remaining_prices_1c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1c) & (Q_B_Preds_bm.index < larger_index_1c)]

                                if len(remaining_prices_1c) > 1:
                                        max_price_index_1c = remaining_prices_1c_A['Price'].idxmax()
                                        min_price_index_1c = remaining_prices_1c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_1c, 'Price'] < current_Q_A_bm.loc[max_price_index_1c, 'Price']:
                                            if min_price_index_1c < max_price_index_1c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_1c > max_price_index_1c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                                        remaining_prices_1d = current_df_bm[(current_df_bm.index > smaller_index_1d) & (current_df_bm.index < larger_index_1d)]
                                        remaining_prices_1d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1d) & (Q_A_Preds_bm.index < larger_index_1d)]
                                        remaining_prices_1d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1d) & (Q_B_Preds_bm.index < larger_index_1d)]
    
                                        if len(remaining_prices_1d) > 1:
                                                max_price_index_1d = remaining_prices_1d_A['Price'].idxmax()
                                                min_price_index_1d = remaining_prices_1d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_1d, 'Price'] < current_Q_A_bm.loc[max_price_index_1d, 'Price']:
                                                    if min_price_index_1d < max_price_index_1d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_1d > max_price_index_1d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_1e = min(min_price_index_1d, max_price_index_1d)
                                                larger_index_1e = max(min_price_index_1d, max_price_index_1d)
                                                remaining_prices_1e = current_df_bm[(current_df_bm.index > smaller_index_1e) & (current_df_bm.index < larger_index_1e)]
                                                remaining_prices_1e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1e) & (Q_A_Preds_bm.index < larger_index_1e)]
                                                remaining_prices_1e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1e) & (Q_B_Preds_bm.index < larger_index_1e)]
    
                                                if len(remaining_prices_1e) > 1:
                                                        max_price_index_1e = remaining_prices_1e_A['Price'].idxmax()
                                                        min_price_index_1e = remaining_prices_1e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_1e, 'Price'] < current_Q_A_bm.loc[max_price_index_1e, 'Price']:
                                                            if min_price_index_1e < max_price_index_1e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_1e > max_price_index_1e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                          
                                        
                                        
                                        
                                        
                                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_2 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_2a = current_df_bm[(current_df_bm.index > smaller_index_2a) & (current_df_bm.index < larger_index_2a)]
                remaining_prices_2a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2a) & (Q_A_Preds_bm.index < larger_index_2a)]
                remaining_prices_2a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2a) & (Q_B_Preds_bm.index < larger_index_2a)]

                if len(remaining_prices_2a) > 1:
                        max_price_index_2a = remaining_prices_2a_A['Price'].idxmax()
                        min_price_index_2a = remaining_prices_2a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_2a, 'Price'] < current_Q_A_bm.loc[max_price_index_2a, 'Price']:
                            if min_price_index_2a < max_price_index_2a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                    
                                profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2a > max_price_index_2a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                    
                                  profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)
                    
                                  prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                        larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                        remaining_prices_2b = current_df_bm[(current_df_bm.index > smaller_index_2b) & (current_df_bm.index < larger_index_2b)]
                        remaining_prices_2b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2b) & (Q_A_Preds_bm.index < larger_index_2b)]
                        remaining_prices_2b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2b) & (Q_B_Preds_bm.index < larger_index_2b)]

                        if len(remaining_prices_2b) > 1:
                                max_price_index_2b = remaining_prices_2b_A['Price'].idxmax()
                                min_price_index_2b = remaining_prices_2b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_2b, 'Price'] < current_Q_A_bm.loc[max_price_index_2b, 'Price']:
                                    if min_price_index_2b < max_price_index_2b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                    
                                        profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                        prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2b > max_price_index_2b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                    
                                          profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)
                    
                                          prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                                larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                                remaining_prices_2c = current_df_bm[(current_df_bm.index > smaller_index_2c) & (current_df_bm.index < larger_index_2c)]
                                remaining_prices_2c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2c) & (Q_A_Preds_bm.index < larger_index_2c)]
                                remaining_prices_2c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2c) & (Q_B_Preds_bm.index < larger_index_2c)]

                                if len(remaining_prices_2c) > 1:
                                        max_price_index_2c = remaining_prices_2c_A['Price'].idxmax()
                                        min_price_index_2c = remaining_prices_2c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_2c, 'Price'] < current_Q_A_bm.loc[max_price_index_2c, 'Price']:
                                            if min_price_index_2c < max_price_index_2c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                    
                                                profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_2c > max_price_index_2c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                    
                                                  profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)
                    
                                                  prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                                        larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                                        remaining_prices_2d = current_df_bm[(current_df_bm.index > smaller_index_2d) & (current_df_bm.index < larger_index_2d)]
                                        remaining_prices_2d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2d) & (Q_A_Preds_bm.index < larger_index_2d)]
                                        remaining_prices_2d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2d) & (Q_B_Preds_bm.index < larger_index_2d)]
    
                                        if len(remaining_prices_2d) > 1:
                                                max_price_index_2d = remaining_prices_2d_A['Price'].idxmax()
                                                min_price_index_2d = remaining_prices_2d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_2d, 'Price'] < current_Q_A_bm.loc[max_price_index_2d, 'Price']:
                                                    if min_price_index_2d < max_price_index_2d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                    
                                                        profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                        prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_2d > max_price_index_2d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                    
                                                          profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)
                    
                                                          prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_2e = min(min_price_index_2d, max_price_index_2d)
                                                larger_index_2e = max(min_price_index_2d, max_price_index_2d)
                                                remaining_prices_2e = current_df_bm[(current_df_bm.index > smaller_index_2e) & (current_df_bm.index < larger_index_2e)]
                                                remaining_prices_2e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2e) & (Q_A_Preds_bm.index < larger_index_2e)]
                                                remaining_prices_2e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2e) & (Q_B_Preds_bm.index < larger_index_2e)]
    
                                                if len(remaining_prices_2e) > 1:
                                                        max_price_index_2e = remaining_prices_2e_A['Price'].idxmax()
                                                        min_price_index_2e = remaining_prices_2e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_2e, 'Price'] < current_Q_A_bm.loc[max_price_index_2e, 'Price']:
                                                            if min_price_index_2e < max_price_index_2e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                    
                                                                profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_2e > max_price_index_2e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                    
                                                                  profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)
                    
                                                                  prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                         
                                                                      

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_30=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_30_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_30_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_50_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_70_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_70_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_dam_bm=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM,          eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_dam_bm.iloc[:,4:5].values),2))

Quantile with price impacts for 1MW battery pentalty attached

In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2) 
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                         x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                         x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[29015.62]
[29862.52]
[30077.]
[29464.09]
[28925.56]
[30781.05]
[27930.2]
[40661.58]


In [17]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2) 
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                         x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                         x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[5723.45]
[10877.74]
[19266.41]
[24049.79]
[18984.49]
[17131.92]
[3010.52]
[163007.04]


In [23]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_0 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_0 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_0 = remaining_prices_0_A.loc[max_price_index_0, 'P_dam']
            min_price_index_0 = remaining_prices_0_B.loc[min_price_index_0, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_0/2, 'Price'] < current_Q_A.loc[max_price_index_0/2, 'Price']:
                  if min_price_index_0 < max_price_index_0:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                    profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_0 > max_price_index_0:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                        profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_0a = min(min_price_index_0, max_price_index_0)
            larger_index_0a = max(min_price_index_0, max_price_index_0)
            remaining_prices_0a = current_df[(current_df['P_dam'] > smaller_index_0a) & (current_df['P_dam'] < larger_index_0a)]
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0a) & (Q_A_Preds['P_dam'] < larger_index_0a)]
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0a) & (Q_B_Preds['P_dam'] < larger_index_0a)]

            if len(remaining_prices_0a) > 1:

                    max_price_index_0a = remaining_prices_0a_A['Price'].idxmax()
                    min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                    max_price_index_0a = remaining_prices_0a_A.loc[max_price_index_0a, 'P_dam']
                    min_price_index_0a = remaining_prices_0a_B.loc[min_price_index_0a, 'P_dam']
                    if current_Q_B.loc[min_price_index_0a/2, 'Price'] < current_Q_A.loc[max_price_index_0a/2, 'Price']:
                        if min_price_index_0a < max_price_index_0a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                            profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_0a > max_price_index_0a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                              profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue     
                                
                                
                                
                                
                    smaller_index_0b = min(min_price_index_0a, max_price_index_0a)
                    larger_index_0b = max(min_price_index_0a, max_price_index_0a)
                    remaining_prices_0b = current_df[(current_df['P_dam'] > smaller_index_0b) & (current_df['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0b) & (Q_A_Preds['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0b) & (Q_B_Preds['P_dam'] < larger_index_0b)]

                    if len(remaining_prices_0b) > 1:

                            max_price_index_0b = remaining_prices_0b_A['Price'].idxmax()
                            min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                            max_price_index_0b = remaining_prices_0b_A.loc[max_price_index_0b, 'P_dam']
                            min_price_index_0b = remaining_prices_0b_B.loc[min_price_index_0b, 'P_dam']
                            if current_Q_B.loc[min_price_index_0b/2, 'Price'] < current_Q_A.loc[max_price_index_0b/2, 'Price']:
                                if min_price_index_0b < max_price_index_0b:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                    profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                    
                                elif min_price_index_0b > max_price_index_0b:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                      profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                    
                                
                                
 
                            smaller_index_0c = min(min_price_index_0b, max_price_index_0b)
                            larger_index_0c = max(min_price_index_0b, max_price_index_0b)
                            remaining_prices_0c = current_df[(current_df['P_dam'] > smaller_index_0c) & (current_df['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0c) & (Q_A_Preds['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0c) & (Q_B_Preds['P_dam'] < larger_index_0c)]

                            if len(remaining_prices_0c) > 1:

                                    max_price_index_0c = remaining_prices_0c_A['Price'].idxmax()
                                    min_price_index_0c = remaining_prices_0c_B['Price'].idxmin()
                                    max_price_index_0c = remaining_prices_0c_A.loc[max_price_index_0c, 'P_dam']
                                    min_price_index_0c = remaining_prices_0c_B.loc[min_price_index_0c, 'P_dam']
                                    if current_Q_B.loc[min_price_index_0c/2, 'Price'] < current_Q_A.loc[max_price_index_0c/2, 'Price']:
                                        if min_price_index_0c < max_price_index_0c:
                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                            charge_level += bottleneck_1
                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                            charge_level -= bottleneck_2
                                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                            profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                            prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                    
                                        elif min_price_index_0c > max_price_index_0c:
                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                              charge_level -= bottleneck_2
                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                              charge_level += bottleneck_1
                                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                              profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                              prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                                
                                        else:
                                             continue


                                    smaller_index_0d = min(min_price_index_0c, max_price_index_0c)
                                    larger_index_0d = max(min_price_index_0c, max_price_index_0c)
                                    remaining_prices_0d = current_df[(current_df['P_dam'] > smaller_index_0d) & (current_df['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0d) & (Q_A_Preds['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0d) & (Q_B_Preds['P_dam'] < larger_index_0d)]

                                    if len(remaining_prices_0d) > 1:

                                            max_price_index_0d = remaining_prices_0d_A['Price'].idxmax()
                                            min_price_index_0d = remaining_prices_0d_B['Price'].idxmin()
                                            max_price_index_0d = remaining_prices_0d_A.loc[max_price_index_0d, 'P_dam']
                                            min_price_index_0d = remaining_prices_0d_B.loc[min_price_index_0d, 'P_dam']
                                            if current_Q_B.loc[min_price_index_0d/2, 'Price'] < current_Q_A.loc[max_price_index_0d/2, 'Price']:
                                                if min_price_index_0d < max_price_index_0d:
                                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                    charge_level += bottleneck_1
                                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                    charge_level -= bottleneck_2
                                                    x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                    x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                    profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                    prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                    
                                                elif min_price_index_0d > max_price_index_0d:
                                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                      charge_level -= bottleneck_2
                                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                      charge_level += bottleneck_1
                                                      x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                      x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                      profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                      prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                                
                                                else:
                                                     continue



                                            smaller_index_0e = min(min_price_index_0d, max_price_index_0d)
                                            larger_index_0e = max(min_price_index_0d, max_price_index_0d)
                                            remaining_prices_0e = current_df[(current_df['P_dam'] > smaller_index_0e) & (current_df['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0e) & (Q_A_Preds['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0e) & (Q_B_Preds['P_dam'] < larger_index_0e)]

                                            if len(remaining_prices_0e) > 1:

                                                    max_price_index_0e = remaining_prices_0e_A['Price'].idxmax()
                                                    min_price_index_0e = remaining_prices_0e_B['Price'].idxmin()
                                                    max_price_index_0e = remaining_prices_0e_A.loc[max_price_index_0e, 'P_dam']
                                                    min_price_index_0e = remaining_prices_0e_B.loc[min_price_index_0e, 'P_dam']
                                                    if current_Q_B.loc[min_price_index_0e/2, 'Price'] < current_Q_A.loc[max_price_index_0e/2, 'Price']:
                                                        if min_price_index_0e < max_price_index_0e:
                                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                            charge_level += bottleneck_1
                                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                            charge_level -= bottleneck_2
                                                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                            profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                            prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                    
                                                        elif min_price_index_0e > max_price_index_0e:
                                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                              charge_level -= bottleneck_2
                                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                              charge_level += bottleneck_1
                                                              x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                              x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                              profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                              prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                                
                                                        else:
                                                             continue









                                            
        if len(remaining_prices_1) > 1:

                max_price_index_1 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_1 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_1, 'Price'] < current_Q_A_bm.loc[max_price_index_1, 'Price']:
                      if min_price_index_1 < max_price_index_1:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                            profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      elif min_price_index_1 > max_price_index_1:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
                             x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                             x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                             profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                             prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      else:
                            continue                
                        

        
                smaller_index_1a = min(min_price_index_1, max_price_index_1)
                larger_index_1a = max(min_price_index_1, max_price_index_1)
                remaining_prices_1a = current_df_bm[(current_df_bm.index > smaller_index_1a) & (current_df_bm.index < larger_index_1a)]
                remaining_prices_1a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1a) & (Q_A_Preds_bm.index < larger_index_1a)]
                remaining_prices_1a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1a) & (Q_B_Preds_bm.index < larger_index_1a)]

                if len(remaining_prices_1a) > 1:
                        max_price_index_1a = remaining_prices_1a_A['Price'].idxmax()
                        min_price_index_1a = remaining_prices_1a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_1a, 'Price'] < current_Q_A_bm.loc[max_price_index_1a, 'Price']:
                            if min_price_index_1a < max_price_index_1a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1a > max_price_index_1a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                  profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                        larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                        remaining_prices_1b = current_df_bm[(current_df_bm.index > smaller_index_1b) & (current_df_bm.index < larger_index_1b)]
                        remaining_prices_1b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1b) & (Q_A_Preds_bm.index < larger_index_1b)]
                        remaining_prices_1b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1b) & (Q_B_Preds_bm.index < larger_index_1b)]

                        if len(remaining_prices_1b) > 1:
                                max_price_index_1b = remaining_prices_1b_A['Price'].idxmax()
                                min_price_index_1b = remaining_prices_1b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_1b, 'Price'] < current_Q_A_bm.loc[max_price_index_1b, 'Price']:
                                    if min_price_index_1b < max_price_index_1b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_1b > max_price_index_1b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                          prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                                larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                                remaining_prices_1c = current_df_bm[(current_df_bm.index > smaller_index_1c) & (current_df_bm.index < larger_index_1c)]
                                remaining_prices_1c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1c) & (Q_A_Preds_bm.index < larger_index_1c)]
                                remaining_prices_1c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1c) & (Q_B_Preds_bm.index < larger_index_1c)]

                                if len(remaining_prices_1c) > 1:
                                        max_price_index_1c = remaining_prices_1c_A['Price'].idxmax()
                                        min_price_index_1c = remaining_prices_1c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_1c, 'Price'] < current_Q_A_bm.loc[max_price_index_1c, 'Price']:
                                            if min_price_index_1c < max_price_index_1c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_1c > max_price_index_1c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                  profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                  prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                                        remaining_prices_1d = current_df_bm[(current_df_bm.index > smaller_index_1d) & (current_df_bm.index < larger_index_1d)]
                                        remaining_prices_1d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1d) & (Q_A_Preds_bm.index < larger_index_1d)]
                                        remaining_prices_1d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1d) & (Q_B_Preds_bm.index < larger_index_1d)]
    
                                        if len(remaining_prices_1d) > 1:
                                                max_price_index_1d = remaining_prices_1d_A['Price'].idxmax()
                                                min_price_index_1d = remaining_prices_1d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_1d, 'Price'] < current_Q_A_bm.loc[max_price_index_1d, 'Price']:
                                                    if min_price_index_1d < max_price_index_1d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                        profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                        prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_1d > max_price_index_1d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                          profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                          prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_1e = min(min_price_index_1d, max_price_index_1d)
                                                larger_index_1e = max(min_price_index_1d, max_price_index_1d)
                                                remaining_prices_1e = current_df_bm[(current_df_bm.index > smaller_index_1e) & (current_df_bm.index < larger_index_1e)]
                                                remaining_prices_1e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1e) & (Q_A_Preds_bm.index < larger_index_1e)]
                                                remaining_prices_1e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1e) & (Q_B_Preds_bm.index < larger_index_1e)]
    
                                                if len(remaining_prices_1e) > 1:
                                                        max_price_index_1e = remaining_prices_1e_A['Price'].idxmax()
                                                        min_price_index_1e = remaining_prices_1e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_1e, 'Price'] < current_Q_A_bm.loc[max_price_index_1e, 'Price']:
                                                            if min_price_index_1e < max_price_index_1e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                                                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                                profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_1e > max_price_index_1e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                                                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                                  profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                  prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                          
                                        
                                        
                                        
                                        
                                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_2 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                            x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_2a = current_df_bm[(current_df_bm.index > smaller_index_2a) & (current_df_bm.index < larger_index_2a)]
                remaining_prices_2a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2a) & (Q_A_Preds_bm.index < larger_index_2a)]
                remaining_prices_2a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2a) & (Q_B_Preds_bm.index < larger_index_2a)]

                if len(remaining_prices_2a) > 1:
                        max_price_index_2a = remaining_prices_2a_A['Price'].idxmax()
                        min_price_index_2a = remaining_prices_2a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_2a, 'Price'] < current_Q_A_bm.loc[max_price_index_2a, 'Price']:
                            if min_price_index_2a < max_price_index_2a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2a > max_price_index_2a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0                   
                                  profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                        larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                        remaining_prices_2b = current_df_bm[(current_df_bm.index > smaller_index_2b) & (current_df_bm.index < larger_index_2b)]
                        remaining_prices_2b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2b) & (Q_A_Preds_bm.index < larger_index_2b)]
                        remaining_prices_2b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2b) & (Q_B_Preds_bm.index < larger_index_2b)]

                        if len(remaining_prices_2b) > 1:
                                max_price_index_2b = remaining_prices_2b_A['Price'].idxmax()
                                min_price_index_2b = remaining_prices_2b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_2b, 'Price'] < current_Q_A_bm.loc[max_price_index_2b, 'Price']:
                                    if min_price_index_2b < max_price_index_2b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                        profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2b > max_price_index_2b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                          profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                          prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                                larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                                remaining_prices_2c = current_df_bm[(current_df_bm.index > smaller_index_2c) & (current_df_bm.index < larger_index_2c)]
                                remaining_prices_2c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2c) & (Q_A_Preds_bm.index < larger_index_2c)]
                                remaining_prices_2c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2c) & (Q_B_Preds_bm.index < larger_index_2c)]

                                if len(remaining_prices_2c) > 1:
                                        max_price_index_2c = remaining_prices_2c_A['Price'].idxmax()
                                        min_price_index_2c = remaining_prices_2c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_2c, 'Price'] < current_Q_A_bm.loc[max_price_index_2c, 'Price']:
                                            if min_price_index_2c < max_price_index_2c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_2c > max_price_index_2c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                  profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                  prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                                        larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                                        remaining_prices_2d = current_df_bm[(current_df_bm.index > smaller_index_2d) & (current_df_bm.index < larger_index_2d)]
                                        remaining_prices_2d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2d) & (Q_A_Preds_bm.index < larger_index_2d)]
                                        remaining_prices_2d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2d) & (Q_B_Preds_bm.index < larger_index_2d)]
    
                                        if len(remaining_prices_2d) > 1:
                                                max_price_index_2d = remaining_prices_2d_A['Price'].idxmax()
                                                min_price_index_2d = remaining_prices_2d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_2d, 'Price'] < current_Q_A_bm.loc[max_price_index_2d, 'Price']:
                                                    if min_price_index_2d < max_price_index_2d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                                                        x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                        x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                        profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                        prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_2d > max_price_index_2d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                                                          x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                          x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                          profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                          prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_2e = min(min_price_index_2d, max_price_index_2d)
                                                larger_index_2e = max(min_price_index_2d, max_price_index_2d)
                                                remaining_prices_2e = current_df_bm[(current_df_bm.index > smaller_index_2e) & (current_df_bm.index < larger_index_2e)]
                                                remaining_prices_2e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2e) & (Q_A_Preds_bm.index < larger_index_2e)]
                                                remaining_prices_2e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2e) & (Q_B_Preds_bm.index < larger_index_2e)]
    
                                                if len(remaining_prices_2e) > 1:
                                                        max_price_index_2e = remaining_prices_2e_A['Price'].idxmax()
                                                        min_price_index_2e = remaining_prices_2e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_2e, 'Price'] < current_Q_A_bm.loc[max_price_index_2e, 'Price']:
                                                            if min_price_index_2e < max_price_index_2e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                                                                x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                                x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                                profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_2e > max_price_index_2e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                                                                  x1 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                                                                  x2 = 0.008 if random.uniform(0, 1) <= 0.036 else 0
                    
                                                                  profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                  prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                         
                                                                      

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_30=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_30_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_30_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_50_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_70_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_70_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)

PF_dam_bm=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM,          eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0)
print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_dam_bm.iloc[:,4:5].values),2))

[43691.52]
[41943.06]
[46885.97]
[49751.44]
[48826.75]
[45020.03]
[35088.84]
[99326.68]


Quantile with price impacts for 10MW battery pentalty attached

In [37]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2) 
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                         x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                         x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])




r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
PF_DAM     =run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r,  Q_B_Preds=Y_r,  eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[435034.71]
[419509.5]
[428303.71]
[420861.04]
[404723.82]
[418955.07]
[337724.27]
[530251.92]


In [39]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        remaining_prices_0 = current_df[(current_df.index > smaller_index) & (current_df.index > larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index > larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index > larger_index)]

        remaining_prices_1 = current_df[(current_df.index > smaller_index) & (current_df.index < larger_index)]
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_1_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index) & (Q_B_Preds.index < larger_index)]

        remaining_prices_2 = current_df[(current_df.index < smaller_index) & (current_df.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < smaller_index) & (Q_A_Preds.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < smaller_index) & (Q_B_Preds.index < larger_index)]
        
        
        
        

        if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2) 
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                
                
                                    
      
    
        if len(remaining_prices_0) > 1:
            max_price_index_1 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_1 = remaining_prices_0_B['Price'].idxmin()
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_1, 'Price'] < current_Q_A.loc[max_price_index_1, 'Price']:
                  if min_price_index_1 < max_price_index_1:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                    profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  elif min_price_index_1 > max_price_index_1:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_1, current_df.loc[min_price_index_1, 'Price'], max_price_index_1, current_df.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                  else:
                        continue  

                        
            smaller_index_1a = min(min_price_index_1, max_price_index_1)
            larger_index_1a = max(min_price_index_1, max_price_index_1)
            remaining_prices_11a = current_df[(current_df.index > smaller_index_1a) & (current_df.index < larger_index_1a)]
            remaining_prices_11a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1a) & (Q_A_Preds.index < larger_index_1a)]
            remaining_prices_11a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1a) & (Q_B_Preds.index < larger_index_1a)]

            if len(remaining_prices_11a) > 1:
                max_price_index_1a = remaining_prices_11a_A['Price'].idxmax()
                min_price_index_1a = remaining_prices_11a_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_1a, 'Price'] < current_Q_A.loc[max_price_index_1a, 'Price']:
                    if min_price_index_1a < max_price_index_1a:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                        profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                        prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                    elif min_price_index_1a > max_price_index_1a:
                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                          charge_level -= bottleneck_2
                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                          charge_level += bottleneck_1
                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                          profit = (current_df.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                          prices.append((min_price_index_1a, current_df.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                    else:
                         continue
                            
                            
                            
                            
                smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                remaining_prices_11b = current_df[(current_df.index > smaller_index_1b) & (current_df.index < larger_index_1b)]
                remaining_prices_11b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1b) & (Q_A_Preds.index < larger_index_1b)]
                remaining_prices_11b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1b) & (Q_B_Preds.index < larger_index_1b)]

                if len(remaining_prices_11b) > 1:
                    max_price_index_1b = remaining_prices_11b_A['Price'].idxmax()
                    min_price_index_1b = remaining_prices_11b_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_1b, 'Price'] < current_Q_A.loc[max_price_index_1b, 'Price']:
                        if min_price_index_1b < max_price_index_1b:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                        elif min_price_index_1b > max_price_index_1b:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                        else:
                             continue                           
                            
                            
                            
                    smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                    larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                    remaining_prices_11c = current_df[(current_df.index > smaller_index_1c) & (current_df.index < larger_index_1c)]
                    remaining_prices_11c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1c) & (Q_A_Preds.index < larger_index_1c)]
                    remaining_prices_11c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1c) & (Q_B_Preds.index < larger_index_1c)]

                    if len(remaining_prices_11c) > 1:
                        max_price_index_1c = remaining_prices_11c_A['Price'].idxmax()
                        min_price_index_1c = remaining_prices_11c_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_1c, 'Price'] < current_Q_A.loc[max_price_index_1c, 'Price']:
                            if min_price_index_1c < max_price_index_1c:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1c > max_price_index_1c:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1c, current_df.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                           
                                               
                                        
                                        
                                        
                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                        remaining_prices_11d = current_df[(current_df.index > smaller_index_1d) & (current_df.index < larger_index_1d)]
                        remaining_prices_11d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_1d) & (Q_A_Preds.index < larger_index_1d)]
                        remaining_prices_11d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_1d) & (Q_B_Preds.index < larger_index_1d)]

                        if len(remaining_prices_11d) > 1:
                            max_price_index_1d = remaining_prices_11d_A['Price'].idxmax()
                            min_price_index_1d = remaining_prices_11d_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_1d, 'Price'] < current_Q_A.loc[max_price_index_1d, 'Price']:
                                if min_price_index_1d < max_price_index_1d:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                elif min_price_index_1d > max_price_index_1d:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_1d, current_df.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                 
                            
                            
                            
                            
                            
                            
                            
                            

                  
        if len(remaining_prices_1) > 1:
                max_price_index_2 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_2, 'Price'] < current_Q_A.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                         x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                         profit = (current_df.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_02a = current_df[(current_df.index > smaller_index_2a) & (current_df.index < larger_index_2a)]
                remaining_prices_02a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2a) & (Q_A_Preds.index < larger_index_2a)]
                remaining_prices_02a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2a) & (Q_B_Preds.index < larger_index_2a)]

                if len(remaining_prices_02a) > 1:
                    max_price_index_2a = remaining_prices_02a_A['Price'].idxmax()
                    min_price_index_2a = remaining_prices_02a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_2a, 'Price'] < current_Q_A.loc[max_price_index_2a, 'Price']:
                        if min_price_index_2a < max_price_index_2a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_2a > max_price_index_2a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_2a, current_df.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                    larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                    remaining_prices_02b = current_df[(current_df.index > smaller_index_2b) & (current_df.index < larger_index_2b)]
                    remaining_prices_02b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2b) & (Q_A_Preds.index < larger_index_2b)]
                    remaining_prices_02b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2b) & (Q_B_Preds.index < larger_index_2b)]

                    if len(remaining_prices_02b) > 1:
                        max_price_index_2b = remaining_prices_02b_A['Price'].idxmax()
                        min_price_index_2b= remaining_prices_02b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_2b, 'Price'] < current_Q_A.loc[max_price_index_2b, 'Price']:
                            if min_price_index_2b < max_price_index_2b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2b > max_price_index_2b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_2b, current_df.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                        larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                        remaining_prices_02c = current_df[(current_df.index > smaller_index_2c) & (current_df.index < larger_index_2c)]
                        remaining_prices_02c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2c) & (Q_A_Preds.index < larger_index_2c)]
                        remaining_prices_02c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2c) & (Q_B_Preds.index < larger_index_2c)]

                        if len(remaining_prices_02c) > 1:
                            max_price_index_2c = remaining_prices_02c_A['Price'].idxmax()
                            min_price_index_2c= remaining_prices_02c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_2c, 'Price'] < current_Q_A.loc[max_price_index_2c, 'Price']:
                                if min_price_index_2c < max_price_index_2c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_2c > max_price_index_2c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_2c, current_df.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                            larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                            remaining_prices_02d = current_df[(current_df.index > smaller_index_2d) & (current_df.index < larger_index_2d)]
                            remaining_prices_02d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_2d) & (Q_A_Preds.index < larger_index_2d)]
                            remaining_prices_02d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_2d) & (Q_B_Preds.index < larger_index_2d)]

                            if len(remaining_prices_02d) > 1:
                                max_price_index_2d = remaining_prices_02d_A['Price'].idxmax()
                                min_price_index_2d= remaining_prices_02d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_2d, 'Price'] < current_Q_A.loc[max_price_index_2d, 'Price']:
                                    if min_price_index_2d < max_price_index_2d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2d > max_price_index_2d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_2d, current_df.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue  
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
                                            
        if len(remaining_prices_2) > 1:
                max_price_index_3 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_3 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B.loc[min_price_index_3, 'Price'] < current_Q_A.loc[max_price_index_3, 'Price']:
                      if min_price_index_3 < max_price_index_3:
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      elif min_price_index_3 > max_price_index_3:
                         bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                         charge_level -= bottleneck_2
                         bottleneck_1 = min(capacity - charge_level, ramp_rate)
                         charge_level += bottleneck_1
                         x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                         x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                         profit = (current_df.loc[max_price_index_3, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                         prices.append((min_price_index_3, current_df.loc[min_price_index_3, 'Price'], max_price_index_3, current_df.loc[max_price_index_3, 'Price'], profit, charge_level))
            
                      else:
                            continue
                        
                smaller_index_3a = min(min_price_index_3, max_price_index_3)
                larger_index_3a = max(min_price_index_3, max_price_index_3)
                remaining_prices_03a = current_df[(current_df.index > smaller_index_3a) & (current_df.index < larger_index_3a)]
                remaining_prices_03a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3a) & (Q_A_Preds.index < larger_index_3a)]
                remaining_prices_03a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3a) & (Q_B_Preds.index < larger_index_3a)]

                if len(remaining_prices_03a) > 1:
                    max_price_index_3a = remaining_prices_03a_A['Price'].idxmax()
                    min_price_index_3a = remaining_prices_03a_B['Price'].idxmin()
                    if current_Q_B.loc[min_price_index_3a, 'Price'] < current_Q_A.loc[max_price_index_3a, 'Price']:
                        if min_price_index_3a < max_price_index_3a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                    
                        elif min_price_index_3a > max_price_index_3a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_3a, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_3a, current_df.loc[min_price_index_3a, 'Price'], max_price_index_3a, current_df.loc[max_price_index_3a, 'Price'], profit, charge_level))
                                    
                        else:
                             continue
                                
                    smaller_index_3b = min(min_price_index_3a, max_price_index_3a)
                    larger_index_3b = max(min_price_index_3a, max_price_index_3a)
                    remaining_prices_03b = current_df[(current_df.index > smaller_index_3b) & (current_df.index < larger_index_3b)]
                    remaining_prices_03b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3b) & (Q_A_Preds.index < larger_index_3b)]
                    remaining_prices_03b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3b) & (Q_B_Preds.index < larger_index_3b)]

                    if len(remaining_prices_03b) > 1:
                        max_price_index_3b = remaining_prices_03b_A['Price'].idxmax()
                        min_price_index_3b= remaining_prices_03b_B['Price'].idxmin()
                        if current_Q_B.loc[min_price_index_3b, 'Price'] < current_Q_A.loc[max_price_index_3b, 'Price']:
                            if min_price_index_3b < max_price_index_3b:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                    
                            elif min_price_index_3b > max_price_index_3b:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df.loc[max_price_index_3b, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                        
                                  prices.append((min_price_index_3b, current_df.loc[min_price_index_3b, 'Price'], max_price_index_3b, current_df.loc[max_price_index_3b, 'Price'], profit, charge_level))
                                        
                            else:
                                 continue


                        smaller_index_3c = min(min_price_index_3b, max_price_index_3b)
                        larger_index_3c = max(min_price_index_3b, max_price_index_3b)
                        remaining_prices_03c = current_df[(current_df.index > smaller_index_3c) & (current_df.index < larger_index_3c)]
                        remaining_prices_03c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3c) & (Q_A_Preds.index < larger_index_3c)]
                        remaining_prices_03c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3c) & (Q_B_Preds.index < larger_index_3c)]

                        if len(remaining_prices_03c) > 1:
                            max_price_index_3c = remaining_prices_03c_A['Price'].idxmax()
                            min_price_index_3c= remaining_prices_03c_B['Price'].idxmin()
                            if current_Q_B.loc[min_price_index_3c, 'Price'] < current_Q_A.loc[max_price_index_3c, 'Price']:
                                if min_price_index_3c < max_price_index_3c:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                    
                                elif min_price_index_3c > max_price_index_3c:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_3c, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                      prices.append((min_price_index_3c, current_df.loc[min_price_index_3c, 'Price'], max_price_index_3c, current_df.loc[max_price_index_3c, 'Price'], profit, charge_level))
                                        
                                else:
                                     continue
                                        
                                        
                            smaller_index_3d = min(min_price_index_3c, max_price_index_3c)
                            larger_index_3d = max(min_price_index_3c, max_price_index_3c)
                            remaining_prices_03d = current_df[(current_df.index > smaller_index_3d) & (current_df.index < larger_index_3d)]
                            remaining_prices_03d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > smaller_index_3d) & (Q_A_Preds.index < larger_index_3d)]
                            remaining_prices_03d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > smaller_index_3d) & (Q_B_Preds.index < larger_index_3d)]

                            if len(remaining_prices_03d) > 1:
                                max_price_index_3d = remaining_prices_03d_A['Price'].idxmax()
                                min_price_index_3d= remaining_prices_03d_B['Price'].idxmin()
                                if current_Q_B.loc[min_price_index_3d, 'Price'] < current_Q_A.loc[max_price_index_3d, 'Price']:
                                    if min_price_index_3d < max_price_index_3d:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_3d > max_price_index_3d:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df.loc[max_price_index_3d, 'Price'] * bottleneck_2 * eff_1)*(1-x1)  - ((current_df.loc[min_price_index_3d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                            
                                          prices.append((min_price_index_3d, current_df.loc[min_price_index_3d, 'Price'], max_price_index_3d, current_df.loc[max_price_index_3d, 'Price'], profit, charge_level))
                                        
                                    else:
                                         continue                                            
                                            
                                                                                      
                                            
                                            
                     

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)
PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.95, eff_2=0.95, capacity=10,charge_level=10, ramp_rate=8.5, min_charge_level=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[319833.83]
[186269.99]
[289658.71]
[335680.31]
[269110.2]
[201747.28]
[35744.67]
[1818120.33]


In [41]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import random

num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)
start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)


def run_electricity_strategy(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']        

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        
        
        remaining_prices_0 = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]
        
        remaining_prices_1 = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
        remaining_prices_1_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
        remaining_prices_1_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]
        remaining_prices_2 = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
        remaining_prices_2_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
        remaining_prices_2_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

        if current_Q_B.loc[min_price_index/2, 'Price'] < current_Q_A.loc[max_price_index/2, 'Price']:
            if min_price_index < max_price_index:
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'], max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            elif min_price_index > max_price_index:
                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                charge_level -= bottleneck_2
                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                charge_level += bottleneck_1
                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                profit = (current_df.loc[max_price_index/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                prices.append((min_price_index, current_df.loc[min_price_index/2, 'Price'],  max_price_index, current_df.loc[max_price_index/2, 'Price'], profit, charge_level))
            
            else:
                continue
                
                
                
                               
                        
        if len(remaining_prices_0) > 1:
            max_price_index_0 = remaining_prices_0_A['Price'].idxmax()
            min_price_index_0 = remaining_prices_0_B['Price'].idxmin()            
            max_price_index_0 = remaining_prices_0_A.loc[max_price_index_0, 'P_dam']
            min_price_index_0 = remaining_prices_0_B.loc[min_price_index_0, 'P_dam']
            
            charge_level=charge_level
            if current_Q_B.loc[min_price_index_0/2, 'Price'] < current_Q_A.loc[max_price_index_0/2, 'Price']:
                  if min_price_index_0 < max_price_index_0:
                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                    charge_level += bottleneck_1
                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                    charge_level -= bottleneck_2
                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                    profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                    prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  elif min_price_index_0 > max_price_index_0:
                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                        charge_level -= bottleneck_2
                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                        charge_level += bottleneck_1
                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                        profit = (current_df.loc[max_price_index_0/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                        prices.append((min_price_index_0, current_df.loc[min_price_index_0/2, 'Price'], max_price_index_0, current_df.loc[max_price_index_0/2, 'Price'], profit, charge_level))
            
                  else:
                        continue  
 
                

            smaller_index_0a = min(min_price_index_0, max_price_index_0)
            larger_index_0a = max(min_price_index_0, max_price_index_0)
            remaining_prices_0a = current_df[(current_df['P_dam'] > smaller_index_0a) & (current_df['P_dam'] < larger_index_0a)]
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0a) & (Q_A_Preds['P_dam'] < larger_index_0a)]
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0a) & (Q_B_Preds['P_dam'] < larger_index_0a)]

            if len(remaining_prices_0a) > 1:

                    max_price_index_0a = remaining_prices_0a_A['Price'].idxmax()
                    min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                    max_price_index_0a = remaining_prices_0a_A.loc[max_price_index_0a, 'P_dam']
                    min_price_index_0a = remaining_prices_0a_B.loc[min_price_index_0a, 'P_dam']
                    if current_Q_B.loc[min_price_index_0a/2, 'Price'] < current_Q_A.loc[max_price_index_0a/2, 'Price']:
                        if min_price_index_0a < max_price_index_0a:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                            profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                            prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                    
                        elif min_price_index_0a > max_price_index_0a:
                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                              charge_level -= bottleneck_2
                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                              charge_level += bottleneck_1
                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                              profit = (current_df.loc[max_price_index_0a/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0a/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                              prices.append((min_price_index_0a, current_df.loc[min_price_index_0a/2, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a/2, 'Price'], profit, charge_level))
                                
                        else:
                             continue     
                                
                                
                                
                                
                    smaller_index_0b = min(min_price_index_0a, max_price_index_0a)
                    larger_index_0b = max(min_price_index_0a, max_price_index_0a)
                    remaining_prices_0b = current_df[(current_df['P_dam'] > smaller_index_0b) & (current_df['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0b) & (Q_A_Preds['P_dam'] < larger_index_0b)]
                    remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0b) & (Q_B_Preds['P_dam'] < larger_index_0b)]

                    if len(remaining_prices_0b) > 1:

                            max_price_index_0b = remaining_prices_0b_A['Price'].idxmax()
                            min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                            max_price_index_0b = remaining_prices_0b_A.loc[max_price_index_0b, 'P_dam']
                            min_price_index_0b = remaining_prices_0b_B.loc[min_price_index_0b, 'P_dam']
                            if current_Q_B.loc[min_price_index_0b/2, 'Price'] < current_Q_A.loc[max_price_index_0b/2, 'Price']:
                                if min_price_index_0b < max_price_index_0b:
                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                    charge_level += bottleneck_1
                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                    charge_level -= bottleneck_2
                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                    profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                    
                                elif min_price_index_0b > max_price_index_0b:
                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                      charge_level -= bottleneck_2
                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                      charge_level += bottleneck_1
                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                      profit = (current_df.loc[max_price_index_0b/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0b/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                      prices.append((min_price_index_0b, current_df.loc[min_price_index_0b/2, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b/2, 'Price'], profit, charge_level))
                                
                                else:
                                     continue                                    
                                
                                
 
                            smaller_index_0c = min(min_price_index_0b, max_price_index_0b)
                            larger_index_0c = max(min_price_index_0b, max_price_index_0b)
                            remaining_prices_0c = current_df[(current_df['P_dam'] > smaller_index_0c) & (current_df['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0c) & (Q_A_Preds['P_dam'] < larger_index_0c)]
                            remaining_prices_0c_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0c) & (Q_B_Preds['P_dam'] < larger_index_0c)]

                            if len(remaining_prices_0c) > 1:

                                    max_price_index_0c = remaining_prices_0c_A['Price'].idxmax()
                                    min_price_index_0c = remaining_prices_0c_B['Price'].idxmin()
                                    max_price_index_0c = remaining_prices_0c_A.loc[max_price_index_0c, 'P_dam']
                                    min_price_index_0c = remaining_prices_0c_B.loc[min_price_index_0c, 'P_dam']
                                    if current_Q_B.loc[min_price_index_0c/2, 'Price'] < current_Q_A.loc[max_price_index_0c/2, 'Price']:
                                        if min_price_index_0c < max_price_index_0c:
                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                            charge_level += bottleneck_1
                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                            charge_level -= bottleneck_2
                                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                            profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                            prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                    
                                        elif min_price_index_0c > max_price_index_0c:
                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                              charge_level -= bottleneck_2
                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                              charge_level += bottleneck_1
                                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                              profit = (current_df.loc[max_price_index_0c/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0c/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                              prices.append((min_price_index_0c, current_df.loc[min_price_index_0c/2, 'Price'], max_price_index_0c, current_df.loc[max_price_index_0c/2, 'Price'], profit, charge_level))
                                
                                        else:
                                             continue


                                    smaller_index_0d = min(min_price_index_0c, max_price_index_0c)
                                    larger_index_0d = max(min_price_index_0c, max_price_index_0c)
                                    remaining_prices_0d = current_df[(current_df['P_dam'] > smaller_index_0d) & (current_df['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0d) & (Q_A_Preds['P_dam'] < larger_index_0d)]
                                    remaining_prices_0d_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0d) & (Q_B_Preds['P_dam'] < larger_index_0d)]

                                    if len(remaining_prices_0d) > 1:

                                            max_price_index_0d = remaining_prices_0d_A['Price'].idxmax()
                                            min_price_index_0d = remaining_prices_0d_B['Price'].idxmin()
                                            max_price_index_0d = remaining_prices_0d_A.loc[max_price_index_0d, 'P_dam']
                                            min_price_index_0d = remaining_prices_0d_B.loc[min_price_index_0d, 'P_dam']
                                            if current_Q_B.loc[min_price_index_0d/2, 'Price'] < current_Q_A.loc[max_price_index_0d/2, 'Price']:
                                                if min_price_index_0d < max_price_index_0d:
                                                    bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                    charge_level += bottleneck_1
                                                    bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                    charge_level -= bottleneck_2
                                                    x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                    x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                    profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                    prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                    
                                                elif min_price_index_0d > max_price_index_0d:
                                                      bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                      charge_level -= bottleneck_2
                                                      bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                      charge_level += bottleneck_1
                                                      x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                      x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                      profit = (current_df.loc[max_price_index_0d/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0d/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                      prices.append((min_price_index_0d, current_df.loc[min_price_index_0d/2, 'Price'], max_price_index_0d, current_df.loc[max_price_index_0d/2, 'Price'], profit, charge_level))
                                
                                                else:
                                                     continue



                                            smaller_index_0e = min(min_price_index_0d, max_price_index_0d)
                                            larger_index_0e = max(min_price_index_0d, max_price_index_0d)
                                            remaining_prices_0e = current_df[(current_df['P_dam'] > smaller_index_0e) & (current_df['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index_0e) & (Q_A_Preds['P_dam'] < larger_index_0e)]
                                            remaining_prices_0e_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index_0e) & (Q_B_Preds['P_dam'] < larger_index_0e)]

                                            if len(remaining_prices_0e) > 1:

                                                    max_price_index_0e = remaining_prices_0e_A['Price'].idxmax()
                                                    min_price_index_0e = remaining_prices_0e_B['Price'].idxmin()
                                                    max_price_index_0e = remaining_prices_0e_A.loc[max_price_index_0e, 'P_dam']
                                                    min_price_index_0e = remaining_prices_0e_B.loc[min_price_index_0e, 'P_dam']
                                                    if current_Q_B.loc[min_price_index_0e/2, 'Price'] < current_Q_A.loc[max_price_index_0e/2, 'Price']:
                                                        if min_price_index_0e < max_price_index_0e:
                                                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                            charge_level += bottleneck_1
                                                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                            charge_level -= bottleneck_2
                                                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                            profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                            prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                    
                                                        elif min_price_index_0e > max_price_index_0e:
                                                              bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                              charge_level -= bottleneck_2
                                                              bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                              charge_level += bottleneck_1
                                                              x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                              x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                              profit = (current_df.loc[max_price_index_0e/2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df.loc[min_price_index_0e/2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                              prices.append((min_price_index_0e, current_df.loc[min_price_index_0e/2, 'Price'], max_price_index_0e, current_df.loc[max_price_index_0e/2, 'Price'], profit, charge_level))
                                
                                                        else:
                                                             continue









                                            
        if len(remaining_prices_1) > 1:

                max_price_index_1 = remaining_prices_1_A['Price'].idxmax()
                min_price_index_1 = remaining_prices_1_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_1, 'Price'] < current_Q_A_bm.loc[max_price_index_1, 'Price']:
                      if min_price_index_1 < max_price_index_1:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                            profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      elif min_price_index_1 > max_price_index_1:
                             bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                             charge_level -= bottleneck_2
                             bottleneck_1 = min(capacity - charge_level, ramp_rate)
                             charge_level += bottleneck_1
                             x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                             x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                             profit = (current_df_bm.loc[max_price_index_1, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                             prices.append((min_price_index_1, current_df_bm.loc[min_price_index_1, 'Price'], max_price_index_1, current_df_bm.loc[max_price_index_1, 'Price'], profit, charge_level))
            
                      else:
                            continue                
                        

        
                smaller_index_1a = min(min_price_index_1, max_price_index_1)
                larger_index_1a = max(min_price_index_1, max_price_index_1)
                remaining_prices_1a = current_df_bm[(current_df_bm.index > smaller_index_1a) & (current_df_bm.index < larger_index_1a)]
                remaining_prices_1a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1a) & (Q_A_Preds_bm.index < larger_index_1a)]
                remaining_prices_1a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1a) & (Q_B_Preds_bm.index < larger_index_1a)]

                if len(remaining_prices_1a) > 1:
                        max_price_index_1a = remaining_prices_1a_A['Price'].idxmax()
                        min_price_index_1a = remaining_prices_1a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_1a, 'Price'] < current_Q_A_bm.loc[max_price_index_1a, 'Price']:
                            if min_price_index_1a < max_price_index_1a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_1a > max_price_index_1a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                  profit = (current_df_bm.loc[max_price_index_1a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_1a, current_df_bm.loc[min_price_index_1a, 'Price'], max_price_index_1a, current_df_bm.loc[max_price_index_1a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_1b = min(min_price_index_1a, max_price_index_1a)
                        larger_index_1b = max(min_price_index_1a, max_price_index_1a)
                        remaining_prices_1b = current_df_bm[(current_df_bm.index > smaller_index_1b) & (current_df_bm.index < larger_index_1b)]
                        remaining_prices_1b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1b) & (Q_A_Preds_bm.index < larger_index_1b)]
                        remaining_prices_1b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1b) & (Q_B_Preds_bm.index < larger_index_1b)]

                        if len(remaining_prices_1b) > 1:
                                max_price_index_1b = remaining_prices_1b_A['Price'].idxmax()
                                min_price_index_1b = remaining_prices_1b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_1b, 'Price'] < current_Q_A_bm.loc[max_price_index_1b, 'Price']:
                                    if min_price_index_1b < max_price_index_1b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.008 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.008 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_1b > max_price_index_1b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df_bm.loc[max_price_index_1b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                          prices.append((min_price_index_1b, current_df_bm.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df_bm.loc[max_price_index_1b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_1c = min(min_price_index_1b, max_price_index_1b)
                                larger_index_1c = max(min_price_index_1b, max_price_index_1b)
                                remaining_prices_1c = current_df_bm[(current_df_bm.index > smaller_index_1c) & (current_df_bm.index < larger_index_1c)]
                                remaining_prices_1c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1c) & (Q_A_Preds_bm.index < larger_index_1c)]
                                remaining_prices_1c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1c) & (Q_B_Preds_bm.index < larger_index_1c)]

                                if len(remaining_prices_1c) > 1:
                                        max_price_index_1c = remaining_prices_1c_A['Price'].idxmax()
                                        min_price_index_1c = remaining_prices_1c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_1c, 'Price'] < current_Q_A_bm.loc[max_price_index_1c, 'Price']:
                                            if min_price_index_1c < max_price_index_1c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_1c > max_price_index_1c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                  profit = (current_df_bm.loc[max_price_index_1c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                  prices.append((min_price_index_1c, current_df_bm.loc[min_price_index_1c, 'Price'], max_price_index_1c, current_df_bm.loc[max_price_index_1c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_1d = min(min_price_index_1c, max_price_index_1c)
                                        larger_index_1d = max(min_price_index_1c, max_price_index_1c)
                                        remaining_prices_1d = current_df_bm[(current_df_bm.index > smaller_index_1d) & (current_df_bm.index < larger_index_1d)]
                                        remaining_prices_1d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1d) & (Q_A_Preds_bm.index < larger_index_1d)]
                                        remaining_prices_1d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1d) & (Q_B_Preds_bm.index < larger_index_1d)]
    
                                        if len(remaining_prices_1d) > 1:
                                                max_price_index_1d = remaining_prices_1d_A['Price'].idxmax()
                                                min_price_index_1d = remaining_prices_1d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_1d, 'Price'] < current_Q_A_bm.loc[max_price_index_1d, 'Price']:
                                                    if min_price_index_1d < max_price_index_1d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                        profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                        prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_1d > max_price_index_1d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                          profit = (current_df_bm.loc[max_price_index_1d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                          prices.append((min_price_index_1d, current_df_bm.loc[min_price_index_1d, 'Price'], max_price_index_1d, current_df_bm.loc[max_price_index_1d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_1e = min(min_price_index_1d, max_price_index_1d)
                                                larger_index_1e = max(min_price_index_1d, max_price_index_1d)
                                                remaining_prices_1e = current_df_bm[(current_df_bm.index > smaller_index_1e) & (current_df_bm.index < larger_index_1e)]
                                                remaining_prices_1e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_1e) & (Q_A_Preds_bm.index < larger_index_1e)]
                                                remaining_prices_1e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_1e) & (Q_B_Preds_bm.index < larger_index_1e)]
    
                                                if len(remaining_prices_1e) > 1:
                                                        max_price_index_1e = remaining_prices_1e_A['Price'].idxmax()
                                                        min_price_index_1e = remaining_prices_1e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_1e, 'Price'] < current_Q_A_bm.loc[max_price_index_1e, 'Price']:
                                                            if min_price_index_1e < max_price_index_1e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                                                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                                profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_1e > max_price_index_1e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                                                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                                  profit = (current_df_bm.loc[max_price_index_1e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_1e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                  prices.append((min_price_index_1e, current_df_bm.loc[min_price_index_1e, 'Price'], max_price_index_1e, current_df_bm.loc[max_price_index_1e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                          
                                        
                                        
                                        
                                        
                                        
                       
                        
                        
        if len(remaining_prices_2) > 1:
                max_price_index_2 = remaining_prices_2_A['Price'].idxmax()
                min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                if current_Q_B_bm.loc[min_price_index_2, 'Price'] < current_Q_A_bm.loc[max_price_index_2, 'Price']:
                      if min_price_index_2 < max_price_index_2:
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      elif min_price_index_2 > max_price_index_2:
                            bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                            charge_level -= bottleneck_2
                            bottleneck_1 = min(capacity - charge_level, ramp_rate)
                            charge_level += bottleneck_1
                            x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                            x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
            
                            profit = (current_df_bm.loc[max_price_index_2, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
            
                            prices.append((min_price_index_2, current_df_bm.loc[min_price_index_2, 'Price'], max_price_index_2, current_df_bm.loc[max_price_index_2, 'Price'], profit, charge_level))
            
                      else:
                            continue  
                        
                        
                      
                smaller_index_2a = min(min_price_index_2, max_price_index_2)
                larger_index_2a = max(min_price_index_2, max_price_index_2)
                remaining_prices_2a = current_df_bm[(current_df_bm.index > smaller_index_2a) & (current_df_bm.index < larger_index_2a)]
                remaining_prices_2a_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2a) & (Q_A_Preds_bm.index < larger_index_2a)]
                remaining_prices_2a_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2a) & (Q_B_Preds_bm.index < larger_index_2a)]

                if len(remaining_prices_2a) > 1:
                        max_price_index_2a = remaining_prices_2a_A['Price'].idxmax()
                        min_price_index_2a = remaining_prices_2a_B['Price'].idxmin()
                        if current_Q_B_bm.loc[min_price_index_2a, 'Price'] < current_Q_A_bm.loc[max_price_index_2a, 'Price']:
                            if min_price_index_2a < max_price_index_2a:
                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                charge_level += bottleneck_1
                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                charge_level -= bottleneck_2
                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                    
                            elif min_price_index_2a > max_price_index_2a:
                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                  charge_level -= bottleneck_2
                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                  charge_level += bottleneck_1
                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0                   
                                  profit = (current_df_bm.loc[max_price_index_2a, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2a, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                  prices.append((min_price_index_2a, current_df_bm.loc[min_price_index_2a, 'Price'], max_price_index_2a, current_df_bm.loc[max_price_index_2a, 'Price'], profit, charge_level))
                                
                            else:
                                 continue                         
                        
                        
                        smaller_index_2b = min(min_price_index_2a, max_price_index_2a)
                        larger_index_2b = max(min_price_index_2a, max_price_index_2a)
                        remaining_prices_2b = current_df_bm[(current_df_bm.index > smaller_index_2b) & (current_df_bm.index < larger_index_2b)]
                        remaining_prices_2b_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2b) & (Q_A_Preds_bm.index < larger_index_2b)]
                        remaining_prices_2b_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2b) & (Q_B_Preds_bm.index < larger_index_2b)]

                        if len(remaining_prices_2b) > 1:
                                max_price_index_2b = remaining_prices_2b_A['Price'].idxmax()
                                min_price_index_2b = remaining_prices_2b_B['Price'].idxmin()
                                if current_Q_B_bm.loc[min_price_index_2b, 'Price'] < current_Q_A_bm.loc[max_price_index_2b, 'Price']:
                                    if min_price_index_2b < max_price_index_2b:
                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                        charge_level += bottleneck_1
                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                        charge_level -= bottleneck_2
                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                        profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                        prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                    
                                    elif min_price_index_2b > max_price_index_2b:
                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                          charge_level -= bottleneck_2
                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                          charge_level += bottleneck_1
                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                          profit = (current_df_bm.loc[max_price_index_2b, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2b, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                          prices.append((min_price_index_2b, current_df_bm.loc[min_price_index_2b, 'Price'], max_price_index_2b, current_df_bm.loc[max_price_index_2b, 'Price'], profit, charge_level))
                                
                                    else:
                                         continue                          
                        
                        
                        
                        
                                smaller_index_2c = min(min_price_index_2b, max_price_index_2b)
                                larger_index_2c = max(min_price_index_2b, max_price_index_2b)
                                remaining_prices_2c = current_df_bm[(current_df_bm.index > smaller_index_2c) & (current_df_bm.index < larger_index_2c)]
                                remaining_prices_2c_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2c) & (Q_A_Preds_bm.index < larger_index_2c)]
                                remaining_prices_2c_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2c) & (Q_B_Preds_bm.index < larger_index_2c)]

                                if len(remaining_prices_2c) > 1:
                                        max_price_index_2c = remaining_prices_2c_A['Price'].idxmax()
                                        min_price_index_2c = remaining_prices_2c_B['Price'].idxmin()
                                        if current_Q_B_bm.loc[min_price_index_2c, 'Price'] < current_Q_A_bm.loc[max_price_index_2c, 'Price']:
                                            if min_price_index_2c < max_price_index_2c:
                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                charge_level += bottleneck_1
                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                charge_level -= bottleneck_2
                                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                    
                                            elif min_price_index_2c > max_price_index_2c:
                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                  charge_level -= bottleneck_2
                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                  charge_level += bottleneck_1
                                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                  profit = (current_df_bm.loc[max_price_index_2c, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2c, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                  prices.append((min_price_index_2c, current_df_bm.loc[min_price_index_2c, 'Price'], max_price_index_2c, current_df_bm.loc[max_price_index_2c, 'Price'], profit, charge_level))
                                
                                            else:
                                                 continue                               
                        
                        
                                        smaller_index_2d = min(min_price_index_2c, max_price_index_2c)
                                        larger_index_2d = max(min_price_index_2c, max_price_index_2c)
                                        remaining_prices_2d = current_df_bm[(current_df_bm.index > smaller_index_2d) & (current_df_bm.index < larger_index_2d)]
                                        remaining_prices_2d_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2d) & (Q_A_Preds_bm.index < larger_index_2d)]
                                        remaining_prices_2d_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2d) & (Q_B_Preds_bm.index < larger_index_2d)]
    
                                        if len(remaining_prices_2d) > 1:
                                                max_price_index_2d = remaining_prices_2d_A['Price'].idxmax()
                                                min_price_index_2d = remaining_prices_2d_B['Price'].idxmin()
                                                if current_Q_B_bm.loc[min_price_index_2d, 'Price'] < current_Q_A_bm.loc[max_price_index_2d, 'Price']:
                                                    if min_price_index_2d < max_price_index_2d:
                                                        bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                        charge_level += bottleneck_1
                                                        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                        charge_level -= bottleneck_2
                                                        x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                        x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                        profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                        prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                    
                                                    elif min_price_index_2d > max_price_index_2d:
                                                          bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                          charge_level -= bottleneck_2
                                                          bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                          charge_level += bottleneck_1
                                                          x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                          x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                          profit = (current_df_bm.loc[max_price_index_2d, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2d, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                          prices.append((min_price_index_2d, current_df_bm.loc[min_price_index_2d, 'Price'], max_price_index_2d, current_df_bm.loc[max_price_index_2d, 'Price'], profit, charge_level))
                                
                                                    else:
                                                         continue  
                                        
                                        
                                        
                                                smaller_index_2e = min(min_price_index_2d, max_price_index_2d)
                                                larger_index_2e = max(min_price_index_2d, max_price_index_2d)
                                                remaining_prices_2e = current_df_bm[(current_df_bm.index > smaller_index_2e) & (current_df_bm.index < larger_index_2e)]
                                                remaining_prices_2e_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index_2e) & (Q_A_Preds_bm.index < larger_index_2e)]
                                                remaining_prices_2e_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index_2e) & (Q_B_Preds_bm.index < larger_index_2e)]
    
                                                if len(remaining_prices_2e) > 1:
                                                        max_price_index_2e = remaining_prices_2e_A['Price'].idxmax()
                                                        min_price_index_2e = remaining_prices_2e_B['Price'].idxmin()
                                                        if current_Q_B_bm.loc[min_price_index_2e, 'Price'] < current_Q_A_bm.loc[max_price_index_2e, 'Price']:
                                                            if min_price_index_2e < max_price_index_2e:
                                                                bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                charge_level += bottleneck_1
                                                                bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                charge_level -= bottleneck_2
                                                                x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                                x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                                profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                    
                                                            elif min_price_index_2e > max_price_index_2e:
                                                                  bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
                                                                  charge_level -= bottleneck_2
                                                                  bottleneck_1 = min(capacity - charge_level, ramp_rate)
                                                                  charge_level += bottleneck_1
                                                                  x1 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                                                                  x2 = 0.01 if random.uniform(0, 1) <= 0.3 else 0
                    
                                                                  profit = (current_df_bm.loc[max_price_index_2e, 'Price'] * bottleneck_2 * eff_1)*(1-x1) - ((current_df_bm.loc[min_price_index_2e, 'Price'] * bottleneck_1)/eff_2)*(1+x2)
                    
                                                                  prices.append((min_price_index_2e, current_df_bm.loc[min_price_index_2e, 'Price'], max_price_index_2e, current_df_bm.loc[max_price_index_2e, 'Price'], profit, charge_level))
                                
                                                            else:
                                                                 continue                                         
                                                                      

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])



r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_10_30=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_30_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_30_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_30_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_50_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_70_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_70_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_30_70=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
r_dam_bm_10_90=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
PF_dam_bm=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM,          eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_dam_bm.iloc[:,4:5].values),2))

[702556.73]
[542294.16]
[613914.76]
[654478.27]
[621736.7]
[515783.76]
[353170.28]
[1100614.15]


In [42]:
r_dam_bm_50_50=run_electricity_strategy(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.95, eff_2=0.95, capacity=10,charge_level=1, ramp_rate=8.5, min_charge_level=1)
print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))


[686181.99]
